In [1]:
class Counter:
    """I count. That is all."""
    def __init__(self, initial=0):
        self.value = initial
        
    def increment(self):
        self.value += 1
        
    def get(self):
        return self.value

In [2]:
c = Counter(42)

In [3]:
c.increment()

In [4]:
c.get()

43

Attributes of instance and attributes of class:

In [5]:
class Counter:
    all_counters = []
    
    def __init__(self, initial=0):
        Counter.all_counters.append(self)
        # ...

In [6]:
Counter.some_other_attribute = 42

In [7]:
Counter.all_counters

[]

In [8]:
c = Counter()

In [9]:
Counter.all_counters

In [10]:
c = Counter()

In [11]:
Counter.all_counters

[<__main__.Counter at 0x7f1f14849790>, <__main__.Counter at 0x7f1efffc5310>]

Internal and public attributes:

In [12]:
class Noop:
    some_attribute = 42
    _internal_attribute = []

In [13]:
Noop.some_attribute

42

In [14]:
Noop._internal_attribute

[]

In [15]:
class Noop:
    __very_internal_attribute = []

In [16]:
Noop.__very_internal_attribute

AttributeError: type object 'Noop' has no attribute '__very_internal_attribute'

In [17]:
Noop._Noop__very_internal_attribute  # how to get access

[]

In [18]:
from collections import deque

In [19]:
class MemorizingDict(dict):
    history = deque(maxlen=10)
    
    def set(self, key, value):
        self.history.append(key)
        self[key] = value
        
    def get_history(self):
        return self.history

In [20]:
d = MemorizingDict({"foo": 42})

In [21]:
d.set("baz", 100500)

In [22]:
print(d.get_history())

deque(['baz'], maxlen=10)


In [23]:
d = MemorizingDict()

In [24]:
d.set("boo", 500100)

In [25]:
print(d.get_history())

deque(['baz', 'boo'], maxlen=10)


Default internal attributes:

In [26]:
class Noop:
    """I do nothing at all."""

In [27]:
Noop.__doc__

'I do nothing at all.'

In [28]:
Noop.__name__

'Noop'

In [29]:
Noop.__module__

'__main__'

In [30]:
Noop.__bases__  # parent classes

(object,)

In [31]:
noop = Noop()

In [32]:
noop.__class__

__main__.Noop

In [33]:
noop.__dict__

{}

In [34]:
Noop.__class__

type

In [35]:
Noop.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'I do nothing at all.',
              '__dict__': <attribute '__dict__' of 'Noop' objects>,
              '__weakref__': <attribute '__weakref__' of 'Noop' objects>})

In [36]:
noop.some_attribute = 42

In [37]:
vars(noop)

{'some_attribute': 42}

Slots:

In [38]:
class Noop:
    __slots__ = ["some_attribute"]

In [39]:
noop = Noop()

In [40]:
noop.some_attribute = 42

In [41]:
noop.some_attribute

42

In [42]:
noop.some_other_attribute = 100500

AttributeError: 'Noop' object has no attribute 'some_other_attribute'

Properties: 

In [43]:
from os.path import dirname

In [44]:
class Path:
    def __init__(self, current):
        self.current = current
        
    def __repr__(self):
        return "Path({})".format(self.current)
    
    @property
    def parent(self):
        return Path(dirname(self.current))  # dynamic computation

In [45]:
p = Path("./examples/some_file.txt")

In [46]:
p.parent

Path(./examples)

In [47]:
class BigDataModel:
    def __init__(self):
        self._params = []
        
    @property
    def params(self):
        return self._params
    
    @params.setter
    def params(self, new_params):
        assert all(map(lambda p: p > 0, new_params))
        self._params = new_params
        
    @params.deleter
    def params(self):
        del self._params

In [48]:
model = BigDataModel()

In [49]:
model.params = [0.1, 0.5, 0.4]

In [50]:
model.params

[0.1, 0.5, 0.4]

Inheritance:

In [51]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
class OtherCounter(Counter):
    def get(self):
        return self.value

In [52]:
c = OtherCounter()

In [53]:
c.get()

0

In [54]:
c.value

0

Super:

In [55]:
class Counter:
    all_counters = []
    
    def __init__(self, initial=0):
        self.__class__.all_counters.append(self)
        self.value = initial

class OtherCounter(Counter):
    def __init__(self, initial=0):
        self.initial = initial
        super().__init__(initial)

In [56]:
oc = OtherCounter()

In [57]:
vars(oc)

{'initial': 0, 'value': 0}

Predicate `isinstance`

In [58]:
class A:
    pass

class B(A):
    pass

In [59]:
isinstance(B(), A)

True

In [60]:
class C:
    pass

In [61]:
isinstance(B(), (A, C))

True

In [62]:
isinstance(B(), A) or isinstance(B(), C)

True

Predicate `issubclass`

In [63]:
class A:
    pass

class B(A):
    pass

issubclass(B, A)

True

In [64]:
class C:
    pass

In [65]:
issubclass(B, (A, C))

True

In [66]:
issubclass(B, A) or issubclass(B, C)

True

Mixin:

In [67]:
class ThreadSafeMixin:
    get_lock = ...
    
    def increment(self):
        with self.get_lock():
            super().increment()
    
    def get(self):
        with self.get_lock():
            return super().get()

In [68]:
class ThreadSafeCounter(ThreadSafeMixin, Counter):
    pass

Decorators for classes:

In [69]:
@deco
class Noop:
    pass

NameError: name 'deco' is not defined

In [70]:
class Noop:
    pass

Noop = deco(Noop)

NameError: name 'deco' is not defined

`ThreadSafeMixin` as decorator:

In [71]:
def thread_safe(cls):
    orig_increment = cls.increment
    orig_get = cls.get
    
    def increment(self):
        with self.get_lock():
            orig_increment(self)
    
    def get(self):
        with self.get_lock():
            return orig_get(self)
        
    cls.get_lock = ...
    cls.increment = increment
    cls.get = get
    return cls

Examples of decorators:

In [72]:
import functools

In [73]:
def singleton(cls):
    instance = None
    
    @functools.wraps(cls)
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    
    return inner

In [74]:
@singleton
class Noop:
    "I do nothing at all."

In [75]:
id(Noop())

139771106067664

In [76]:
id(Noop())

139771106067664

In [77]:
import warnings

In [78]:
def deprecated(cls):
    orig_init = cls.__init__
    
    @functools.wraps(cls.__init__)
    def new_init(self, *args, **kwargs):
        warnings.warn(
            cls.__name__ + " is deprecated.",
            category=DeprecationWarning
        )
        orig_init(self, *args, **kwargs)
    
    cls.__init__ = new_init
    return cls

In [79]:
@deprecated
class Counter:
    def __init__(self, initial=0):
        self.value = initial

In [80]:
c = Counter()

/home/leo/Conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Counter is deprecated.
  


### Magic methods

Method `__getattr__` is called in attempt to read the value of non-existing attribute:

In [81]:
class Noop:
    pass

In [82]:
Noop().foobar

AttributeError: 'Noop' object has no attribute 'foobar'

In [83]:
class Noop:
    def __getattr__(self, name):
        return name

In [84]:
Noop().foobar

'foobar'

Methods `__setattr__` and `__delattr__` are used to change values of attribute and delete it:

In [86]:
class Guarded:
    guarded = []
    
    def __setattr__(self, name, value):
        assert name not in self.guarded
        super().__setattr__(name, value)

class Noop(Guarded):
    guarded = ["foobar"]
    
    def __init__(self):
        self.__dict__["foobar"] = 42

In [92]:
class Noop:
    some_attribute = 42

In [93]:
noop = Noop()

In [94]:
getattr(noop, "some_attribute")

42

In [96]:
getattr(noop, "some_other_attribute", 100500)

100500

In [99]:
setattr(noop, "some_other_attribute", 100500)

In [100]:
delattr(noop, "some_other_attribute")

Comparison operators:

In [102]:
@functools.total_ordering
class Counter:
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value

Method `__call__` allows calling instances of classes hence imitating functional interface:

In [103]:
class Identity:
    def __call__(self, x):
        return x

In [104]:
Identity()(42)

42

In [105]:
class trace:
    def __init__(self, handle):
        self.handle = handle
        
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=self.handle)
            return func(*args, **kwargs)
        return inner

In [107]:
import sys


@trace(sys.stderr)
def identity(x):
    return x

In [108]:
identity(42)

identity (42,) {}


42

`repr` and `str`:

In [109]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __repr__(self):
        return "Counter({})".format(self.value)

    def __str__(self):
        return "Counted to {}".format(self.value)

In [111]:
c = Counter(42)

In [113]:
c

Counter(42)

In [114]:
print(c)

Counted to 42


`__format__`:

In [115]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __format__(self, format_spec):
        return self.value.__format__(format_spec)

In [116]:
c = Counter(42)

In [117]:
"Counted to {:b}".format(c)

'Counted to 101010'

`__hash__`

`__bool__`:

In [119]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __bool__(self):
        return bool(self.value)

In [120]:
c = Counter()

In [121]:
if not c:
    print("No counts yet.")

No counts yet.
